# Notebook helper to create Redshift tables

In [1]:
%load_ext sql

In [2]:
import boto3
import configparser
import pandas as pd

#### Set verbosity to 0|1|2 (default=0)

In [3]:
VERBOSE = 1

## STEP-1: Load cluster parameters from `myDWH.cfg`

In [4]:
config = configparser.ConfigParser()
config.read_file(open('../myDWH.cfg'))

# Retrieve the USER-related  parameters
USR_KEY                = config.get('USR','USR_KEY')
USR_SECRET             = config.get('USR','USR_SECRET')

# Retrieve the AWS-related  parameters
AWS_REGION             = config.get('AWS','AWS_REGION')

# Retrieve the CLUSTER-related  parameters
CLUSTER_NAME           = config.get("CLUSTER","CLUSTER_NAME")
CLUSTER_TYPE           = config.get("CLUSTER","CLUSTER_TYPE")
CLUSTER_NODE_TYPE      = config.get("CLUSTER","CLUSTER_NODE_TYPE")
CLUSTER_NODE_COUNT     = config.get("CLUSTER","CLUSTER_NODE_COUNT")

CLUSTER_DB_NAME        = config.get("CLUSTER","CLUSTER_DB_NAME")
CLUSTER_DB_USER        = config.get("CLUSTER","CLUSTER_DB_USER")
CLUSTER_DB_PASSWORD    = config.get("CLUSTER","CLUSTER_DB_PASSWORD")
CLUSTER_DB_PORT        = config.get("CLUSTER","CLUSTER_DB_PORT")

# Retrieve the IAM-related parameters
AWS_RESOURCE_NAME      = config.get("IAM_ROLE", "AWS_RESOURCE_NAME")

(CLUSTER_DB_USER, CLUSTER_DB_PASSWORD, CLUSTER_DB_NAME)

if VERBOSE > 0:
    df = pd.DataFrame({"Param":
                       ["CLUSTER_TYPE", "CLUSTER_NODE_COUNT", "CLUSTER_NODE_TYPE", "CLUSTER_NAME", "CLUSTER_DB_NAME", "CLUSTER_DB_USER", "CLUSTER_DB_PASSWORD", "CLUSTER_DB_PORT", "AWS_RESOURCE_NAME", "AWS_REGION"],
                       "Value":
                       [ CLUSTER_TYPE ,  CLUSTER_NODE_COUNT ,  CLUSTER_NODE_TYPE ,  CLUSTER_NAME ,  CLUSTER_DB_NAME ,  CLUSTER_DB_USER ,  CLUSTER_DB_PASSWORD ,  CLUSTER_DB_PORT ,  AWS_RESOURCE_NAME ,  AWS_REGION ]
                      })
    print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'myDWH.cfg'

## STEP-2: Create a client for Redshift and IAM

In [ ]:
if VERBOSE > 0:
    print("Create clients for Redshift and IAM")

redshift = boto3.client('redshift',
                       region_name=AWS_REGION,
                       aws_access_key_id=USR_KEY,
                       aws_secret_access_key=USR_SECRET
                       )

iam = boto3.client('iam',aws_access_key_id=USR_KEY,
                     aws_secret_access_key=USR_SECRET,
                     region_name=AWS_REGION
                  )

## STEP-3: Retrieve ENDPOINT and IAM Role ARN 

In [ ]:
# from Redshift.Client.exceptions import ClusterNotFound
from botocore.exceptions import ClientError

try:
    # Describe the created cluster
    cluster_props = redshift.describe_clusters(ClusterIdentifier=CLUSTER_NAME)['Clusters'][0]
except ClientError as ce:
    if ce.response['Error']['Code'] == 'ClusterNotFound':
        print("ERROR: The cluster \'%s\' does not exist!" % CLUSTER_NAME)
    else:
        print("Unexpected error: %s" % ce)
else:
    CLUSTER_ENDPOINT = cluster_props['Endpoint']['Address']
    CLUSTER_ROLE_ARN = cluster_props['IamRoles'][0]['IamRoleArn']
    if VERBOSE > 0:
        # Display the cluster endpoint and role ARN 
        print("\nFYI: This is the created cluster ENDPOINT and cluster Role ARN")
        print("\tCLUSTER_ENDPOINT :: ", CLUSTER_ENDPOINT)
        print("\tCLUSTER_ROLE_ARN :: ", CLUSTER_ROLE_ARN)
        

## STEP-4: Connect to Retrieve ENDPOINT and IAM Role ARN 